In [11]:
import networkx as nx
import matplotlib.pyplot as plt

from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram

import numpy as np
from scipy.optimize import minimize

In [12]:
GateX = np.array([[0, 1],[1, 0]])
GateY = np.array([[0, -1j],[1j, 0]])
GateZ = np.array([[1, 0],[0, -1]])

In [13]:
# dtype=np.complex128
def add_in_target(num_qubits, target_qubit, gate=np.array([[1, 0],[0, -1]])):
    H = np.eye(2 ** (target_qubit))
    H = np.kron(H, gate)
    H = np.kron(H, np.eye(2 ** (num_qubits - 1 - target_qubit)))
    return H
    
def gnrt_Hd():
  Hd = np.zeros((2**3, 2**3)).astype(np.complex128)
  Hd += (add_in_target(3, 0, GateX) @ add_in_target(3, 1, GateX) + add_in_target(3, 0, GateY) @ add_in_target(3, 1, GateY))
  Hd += (add_in_target(3, 1, GateX) @ add_in_target(3, 2, GateX) - add_in_target(3, 1, GateY) @ add_in_target(3, 2, GateY))
  Hd += (add_in_target(3, 2, GateX) @ add_in_target(3, 0, GateX) - add_in_target(3, 2, GateY) @ add_in_target(3, 0, GateY))
  # Hd += add_in_target(1, 0, GateX)
  # Hd += add_in_target(3, 1, GateX)
  # Hd += add_in_target(3, 2, GateX)
  return Hd

In [14]:
B = np.zeros(2**2)
B[0] = 1
B

array([1., 0., 0., 0.])

In [15]:
Hd = np.zeros((2**2, 2**2)).astype(np.complex128)
Hd + add_in_target(2, 1, GateX)

array([[0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j]])

In [16]:
add_in_target(2, 0, GateX)


array([[0., 0., 1., 0.],
       [0., 0., 0., 1.],
       [1., 0., 0., 0.],
       [0., 1., 0., 0.]])

In [17]:
A = gnrt_Hd()
print(A)
eigenvalues, eigenvectors = np.linalg.eig(A)
print("Eigenvalues:")
print(eigenvalues)
print("\nEigenvectors:")
print(eigenvectors)

[[0.+0.j 0.+0.j 0.+0.j 2.+0.j 0.+0.j 2.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 2.+0.j 0.+0.j 0.+0.j 2.+0.j]
 [2.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 2.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 2.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 2.+0.j]
 [2.+0.j 0.+0.j 0.+0.j 2.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 2.+0.j 0.+0.j 2.+0.j 0.+0.j 0.+0.j 0.+0.j]]
Eigenvalues:
[-2.+0.j  4.+0.j -2.+0.j  4.+0.j -2.+0.j -2.+0.j  0.+0.j  0.+0.j]

Eigenvectors:
[[ 0.81649658+0.j  0.57735027+0.j  0.42493167-0.j  0.21551589-0.j
   0.26657948+0.j -0.1587708 +0.j  0.        +0.j  0.        +0.j]
 [-0.        -0.j -0.        -0.j -0.        -0.j -0.        -0.j
   0.        +0.j  0.        +0.j  1.        +0.j  0.        +0.j]
 [-0.        -0.j -0.        -0.j -0.34860427-0.j  0.53561762+0.j
   0.76513302+0.j -0.10618901+0.j  0.        +0.j  0.        +0.j]
 [-0.40824829-0.j  0.57735027-0.j 

In [18]:
A = gnrt_Hd()

# 求解本征值和本征态
eigenvalues, eigenvectors = np.linalg.eig(A)

# 输出结果
print("Eigenvalues:")
print(eigenvalues)
print("\nEigenvectors:")
print(eigenvectors[:, 1])

Eigenvalues:
[-2.+0.j  4.+0.j -2.+0.j  4.+0.j -2.+0.j -2.+0.j  0.+0.j  0.+0.j]

Eigenvectors:
[ 0.57735027+0.j -0.        -0.j -0.        -0.j  0.57735027-0.j
 -0.        -0.j  0.57735027-0.j -0.        -0.j -0.        -0.j]


In [19]:
from qiskit import QuantumCircuit
import numpy as np

# 创建一个2量子比特的电路
qc = QuantumCircuit(2)

# 定义一个初始量子态
initial_state = np.array([1/np.sqrt(2), 1j/np.sqrt(2)])

# 使用initialize()方法将电路初始化为给定的量子态
qc.initialize(initial_state, [0])

# 绘制量子电路
print(qc.draw())


     ┌──────────────────────────────┐
q_0: ┤ Initialize(0.70711,0.70711j) ├
     └──────────────────────────────┘
q_1: ────────────────────────────────
                                     


In [20]:
import numpy as np

# 定义一个矩阵
matrix  = np.random.rand(2, 2)

# 计算特征值和特征向量
eigenvalues, eigenvectors = np.linalg.eig(matrix)

# 将特征值和特征向量按照特征值的大小进行排序
sorted_indices = np.argsort(eigenvalues)
sorted_eigenvalues = eigenvalues[sorted_indices]
sorted_eigenvectors = eigenvectors[:, sorted_indices]

# 输出特征值和对应的特征向量
for i in range(len(sorted_eigenvalues)):
    print("特征值:", sorted_eigenvalues[i])
    print("对应的特征向量:", sorted_eigenvectors[:, i])


特征值: -0.34803434698855407
对应的特征向量: [-0.90971228  0.41523917]
特征值: 1.1523409592484297
对应的特征向量: [-0.66327987 -0.74837144]
